### Extra Method: RNN - Recurrent Neural Networks

RNNs are a type of neural network with known capabilities for handling and making predictions on sequential and time-series data. This is due to their bi-directional nature, which contrasts with the uni-directional feed-forward neural networks that are often used as the first example when venturing into deep learning.  
One particularly well-known RNN used for such sequential-data problems is the LSTM or 'Long short-term memory' neural network, it has some distinct structures in its design that allow the network to have a kind of selective latent memory. The specifics of this aren't explored here, but can be read about in greater detail. [1]  

Below is an implementation of LSTMs using Tensorflow, it's used to make train models on the data, and the RMSE is compared to the previous models. The method is explained in comments as written.

#### LSTM for Time-Series Prediction

In [72]:
# Import necessary library functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

def train_lstm(df, column, p, lstm_blocks=4, epochs=50, dense=1, verbose=0):
        # Sensitive to unscaled data, so first min-max scaling if required
        # Nb this will mess up RMSE comparisons, so predictions are reverse engineered after training
        temp = df.copy()
        series = temp[column]
        
        temp['y']  = np.array((series - min(series)) / (max(series) - min(series))) 
        temp['x'] = temp['y']
        
        # The LSTM's X-variables are the lagged information, and the target Y-variable, the current one,
        # like an autoregressive model
        X = np.empty((len(series), p))
        for i in range(p):
            temp['x'] = temp['x'].shift()
            X[:, i] = temp['x']
        
        #Remove rows that will contain incomplete data/NaNs from shift function
        X = X[p:, :]    
        y = np.array(temp['y'])[p:]
        
        # Must reshape to fit keras' [sample, steps, features] array format
        X = np.reshape(X, (X.shape[0], 1, p))
        
        # NN construction
        nn = Sequential()
        #LSTM Layers
        nn.add(LSTM(lstm_blocks, input_shape=(1, p)))
        #Normal hidden layers
        nn.add(Dense(dense))
        nn.compile(loss='mean_squared_error', optimizer='adam')
        results = nn.fit(X, y, epochs=epochs, batch_size=1, verbose=verbose)
        # Display final RMSE from minimised loss function
        print("Final Model Loss MSE: {}".format(results.history['loss'][-1]))
        
        preds = nn.predict(X)
        # Must reverse engineer original values post scaling 
        predlist = [np.nan]*p
        preds_rescaled = preds * (max(series) - min(series)) + min(series)
        [predlist.append(val) for val in preds_rescaled]
        temp['preds'] = predlist 
        
        return temp

In [73]:
results = train_lstm(oj, 'Close', 4, epochs=500)
print("LSTM RMSE: {}".format(eval_fcast(results, 'Close', 'preds')))
eval_matrix.loc['Orange Juice Ticker', :]

Final Model Loss MSE: 0.0014851029263809323
LSTM RMSE: [4.9016814]


Naive Forecast                            4.433167
4-Point Moving Average                    6.323386
4-Point Moving Average of Differences     5.096058
Seasonal Naive                           10.032894
Name: Orange Juice Ticker, dtype: float64

With a relatively small amount of training, the LSTM neural network produces results with an RMSE slightly below; however with a larger number of epochs, this value is likely to improve. Epochs are essentially iterations off passes over the training data by the neural network, the more epochs, the more training, and the more the model fits to the training data's function.  Here is a second example with an increased number of epochs performed:

In [74]:
longer = train_lstm(oj, 'Close', 4, epochs=2000)
print("LSTM RMSE: {}".format(eval_fcast(longer, 'Close', 'preds')))

Final Model Loss MSE: 0.0014920104295015335
LSTM RMSE: [4.328945]


Running on all datasets and comparing to basic methods:

In [75]:
nikkei = train_lstm(nikkei, 'Close', 4, epochs=500)
house = train_lstm(house, 'price', 4, epochs=500)
temps = train_lstm(temps, 'Temperature', 4, epochs=500)
ada = train_lstm(ada, 'Close', 4, epochs=500)
jpy = train_lstm(jpy_cny, 'Close', 4, epochs=500)

Final Model Loss MSE: 0.0033828800078481436
Final Model Loss MSE: 0.021850088611245155
Final Model Loss MSE: 0.00500189745798707
Final Model Loss MSE: 0.0007401913753710687
Final Model Loss MSE: 0.0003037355199921876


In [76]:
lstm_results = [eval_fcast(longer, 'Close', 'preds')[0]]
[lstm_results.append(eval_fcast(d, 'Close', 'preds')[0]) for d in [ada, jpy, nikkei]]
lstm_results.append(eval_fcast(house, 'price', 'preds')[0])
lstm_results.append(eval_fcast(temps, 'Temperature', 'preds')[0])
 
eval_matrix['LSTM'] = lstm_results
eval_matrix

,Naive Forecast,4-Point Moving Average,4-Point Moving Average of Differences,Seasonal Naive,LSTM
Orange Juice Ticker,4.433167,6.323386,5.096058,10.032894,4.328945
ADA GBP,0.053271,0.068681,0.060013,0.109874,0.055592
JPY CNY,0.000332,0.000423,0.000371,0.000689,0.000346
Nikkei,342.512712,461.762239,384.041281,752.020005,342.395752
House Prices,124991.752850,103019.161375,142487.876067,120252.603191,89277.671875
NI Temps,2.616180,4.843154,2.969773,1.673777,1.359446


Comparing with the basic forecast methods, we can see the RMSEs for the majority of the LSTM models is actually below the naive forecast benchmark, and these were relatively undertrained models. With a larger number of epochs and more fine tuning of the structures and input variables, these results may be improved even further.  
The more difficult to predict series appear to be the currency exchange ratios, which behave more unpredictably, and are likely to be more affected by real-world fundamental geopolitical factors, rather than purely mathematical procedures.

#### Sources, References, Libraries
[1] - LSTM: https://www.geeksforgeeks.org/understanding-of-lstm-networks/

numpy - https://numpy.org/  
pandas - https://pandas.pydata.org/  
statsmodels - https://www.statsmodels.org/stable/index.html  
keras - https://keras.io/  